In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


X_train, X_test, y_train, y_test = train_test_split(features, 
                                                target, 
                                                test_size=0.25, 
                                                random_state=281122, 
                                                stratify=(target)) #стратификация выборок по целевому признаку

categorical_encoder = OneHotEncoder(drop="first") # для категориальных
numeric_transformer = StandardScaler() # для количественных

#     создаем пайплайн для кодирования
col_transformer = ColumnTransformer(
    [   # тупо название / оhe                 #, categorical тут у меня   
        ("categorical", categorical_transformer, ["bla", "bla1", "bla2"]), # я еще в начале проекта сделал списки с названиями 
        # название  / скеллер          # а тут , numeric                   # колонок categorical и numeric и задавал эти переменные
        ("numeric", num_transformer, ["notbla", "fkj", "skdfjk"])
    ])
#     создаем пайплайн для грид сёрч
pl = Pipeline(
    [     # название    # предыдущий "пайплайн для кодирования, где охе и скелер"
        ("transformer", col_transformer),
        ("model", model) # тут ясно
    ])
#     кроссвалидация с подбором параметров
grid_search = GridSearchCV(pl, params, cv=5, scoring="roc_auc", n_jobs=-1) # пихаем пайплайн в гридсерч вместо модели
model = grid_search.fit(X_train, y_train)
#     в переменной модел лежит весь пайплайн в виде списка из column_transformera и модели
      # model[0] - кодировщик
      # model[1] - модель


# примечания!!!!!

# такие гиперпараметры:         # ибо можно задавать гиперы и для других участников пайплайна
lgbm = LGBMClassifier(random_state=281122, silent=True, class_weight="balanced")
lgbm_hyper = {"max_depth": range(10, 26), 
              "n_estimators": range(100, 300, 25), 
              "learning_rate": [0.05, 0.1]}  
# для пайплайна задаются вот так:   
lgbm = LGBMClassifier(random_state=281122, silent=True, class_weight="balanced")
lgbm_hyper = {"model__max_depth": range(10, 26), 
              "model__n_estimators": range(100, 300, 25), 
              "model__learning_rate": [0.05, 0.1]}           
# перед двойным андеоскором то название, которое ты указал для модели при объявлении пайплайна
# и вот у тебя есть лучшая необученная модель, точнее лучший пайплайн. пихаем его в переменную как обычно
best_model = model.best_estimator_
# обучаем
best_model.fit(X_train, y_train)
# теперь, чтобы закодировать тестовую выборку берешь нулевой индекс, чтобы предсказывать - первый
X_test = best_model[0].transform(X_test)
positive_predictions = best_model[1].predict_proba(X_test)[:, 1]
# и дальше считаешь рок-аук

NameError: name 'features' is not defined